In [2]:
import pandas as pd
import numpy as np

In [3]:
winds_df = pd.read_csv('../datasets/tiny-imagenet-200/wnids.txt', sep=" ", header=None)
winds_df.columns = ["winds"]

In [4]:
words_winds_df = pd.read_csv('../datasets/tiny-imagenet-200/words.txt', sep="\t", header=None)
words_winds_df.columns = ["winds", "classes"]

In [6]:
tiny_imagenet_classes = pd.merge(words_winds_df, winds_df, how='inner', on=['winds'])

In [7]:
tiny_imagenet_classes.head()

,winds,classes
0,n01443537,"goldfish, Carassius auratus"
1,n01629819,"European fire salamander, Salamandra salamandra"
2,n01641577,"bullfrog, Rana catesbeiana"
3,n01644900,"tailed frog, bell toad, ribbed toad, tailed to..."
4,n01698640,"American alligator, Alligator mississipiensis"


In [9]:
tiny_imagenet_classes.to_csv('../datasets/tiny-imagenet-200/class_labels.csv', index=False)